In [1]:
import pandas as pd
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar
from camel_tools.tokenizers.word import simple_word_tokenize
from collections import Counter
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from nexus_ai.sentence_sentiment_analysis import multi_class_model
from nexus_ai.sentence_sentiment_analysis import model
from nexus_ai.sentence_sentiment_analysis.sentence_sentiment_analysis_model import train, test, predict
import numpy as np
from nexus_ai.sentence_sentiment_analysis.preprocessing import clean_reviews, creat_vocab, tokenize_data, pad_features, train_test_split, google_clean_reviews, clean_reviews_and_creat_vocab

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\meshari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('dataset/ar_reviews_100k.tsv',sep='\t')
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [3]:
print(f'number of null values \n{df.isnull().sum()}')
print(f'number of duplicate values is {df.duplicated().sum()}')

number of null valueslabel    0
text     0
dtype: int64
number of duplicate values0


In [4]:
labels = df['label'].unique()
print(f'labels {labels}')

labels ['Positive' 'Mixed' 'Negative']


let's tranform Mixed into Neutral

In [5]:
print(f'dataset shape before transforming: {df.shape}')
df.loc[df[df['label']=='Mixed'].index, 'label'] = 'Neutral'
# print(len((df[df['label']=='Mixed']).index.to_list()))
# print(df.loc[df[(df['label']=='Mixed')].index, :])
print(f'dataset shape after transforming: {df.shape}')
labels = df['label'].unique()
print(f'labels {labels}')

dataset shape before transforming: (99999, 2)
dataset shape after transforming: (99999, 2)
labels ['Positive' 'Neutral' 'Negative']


In [6]:
pos_prec = df[df['label'] == 'Positive'].shape[0]
neg_prec = df[df['label'] == 'Negative'].shape[0]
nat_prec = df[df['label'] == 'Neutral'].shape[0]
print(f'Positive precentage {pos_prec}')
print(f'Negative precentage {neg_prec}')
print(f'Neutral precentage {nat_prec}')

Positive precentage 33333
Negative precentage 33333
Neutral precentage 33333


the dataset is balanced which is very good

now let's use the camel tools to preproccess the data

In [7]:
print('before unicode normalizing')
print(df.loc[:3, 'text'].to_list())
df['text'] = df['text'].apply(lambda x: normalize_unicode(x))

print('after unicode normalizing')
print(df.loc[:3, 'text'].to_list())

before unicode normalizing
['ممتاز نوعا ما . النظافة والموقع والتجهيز والشاطيء. المطعم', 'أحد أسباب نجاح الإمارات أن كل شخص في هذه الدولة يعشق ترابها. نحن نحب الإمارات. ومضات من فكر. نصائح لدولة تطمح بالصفوف الأولى و قائد لا يقبل إلا براحة شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءة مرات كثيرة', 'هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى هدوء جبال الشيشان .. للتعرف على حقيقة ما يجرى فى تلك البلاد من حروب ضاربة بحق المسلمين و جزء كبير من تاريخ تلك المنطقة. التضحية .. الرجولة .. الوفاء والكثير من القيم الأخرى اثبتت وجودها فى تلك الرواية البسيطة', 'خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطى مرحلة ان القارئ يخلص الرواية وهو فاتح بؤه لمرحلة ان القارئ يخلص الرواية وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبة ف اوائل القرن العشرين وجه ياخدنا لهناك.. خلطة مشاعر انسانية حقيقية لدرجة غريبة.. دراما نقلته من كاتب شاب بيستعرض لصنايعي حقيق

In [8]:
print('oringinal text')
print(df.loc[:3, 'text'].to_list())


df['text'] = df['text'].apply(lambda x: normalize_alef_ar(x))
print('text after Normalizing alef variants to (ا)')
print(df.loc[:3, 'text'].to_list())


df['text'] = df['text'].apply(lambda x: normalize_alef_maksura_ar(x))
print('text after Normalizing alef maksura (ى) to yeh (ي)')
print(df.loc[:3, 'text'].to_list())

df['text'] = df['text'].apply(lambda x: normalize_teh_marbuta_ar(x))
print('text after Normalizing teh marbuta (ة) to heh (ه)')
print(df.loc[:3, 'text'].to_list())

oringinal text
['ممتاز نوعا ما . النظافة والموقع والتجهيز والشاطيء. المطعم', 'أحد أسباب نجاح الإمارات أن كل شخص في هذه الدولة يعشق ترابها. نحن نحب الإمارات. ومضات من فكر. نصائح لدولة تطمح بالصفوف الأولى و قائد لا يقبل إلا براحة شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءة مرات كثيرة', 'هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى هدوء جبال الشيشان .. للتعرف على حقيقة ما يجرى فى تلك البلاد من حروب ضاربة بحق المسلمين و جزء كبير من تاريخ تلك المنطقة. التضحية .. الرجولة .. الوفاء والكثير من القيم الأخرى اثبتت وجودها فى تلك الرواية البسيطة', 'خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطى مرحلة ان القارئ يخلص الرواية وهو فاتح بؤه لمرحلة ان القارئ يخلص الرواية وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبة ف اوائل القرن العشرين وجه ياخدنا لهناك.. خلطة مشاعر انسانية حقيقية لدرجة غريبة.. دراما نقلته من كاتب شاب بيستعرض لصنايعي حقيقي تثق فيما س

In [9]:
print('oringinal text')
print(df.loc[:3, 'text'].to_list())


df['text'] = df['text'].apply(lambda x: dediac_ar(x))
print('text after removing Arabic diacritical marks')
print(df.loc[:3, 'text'].to_list())

oringinal text
['ممتاز نوعا ما . النظافه والموقع والتجهيز والشاطيء. المطعم', 'احد اسباب نجاح الامارات ان كل شخص في هذه الدوله يعشق ترابها. نحن نحب الامارات. ومضات من فكر. نصائح لدوله تطمح بالصفوف الاولي و قائد لا يقبل الا براحه شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءه مرات كثيره', 'هادفه .. وقويه. تنقلك من صخب شوارع القاهره الي هدوء جبال الشيشان .. للتعرف علي حقيقه ما يجري في تلك البلاد من حروب ضاربه بحق المسلمين و جزء كبير من تاريخ تلك المنطقه. التضحيه .. الرجوله .. الوفاء والكثير من القيم الاخري اثبتت وجودها في تلك الروايه البسيطه', 'خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطي مرحله ان القارئ يخلص الروايه وهو فاتح بؤه لمرحله ان القارئ يخلص الروايه وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبه ف اوائل القرن العشرين وجه ياخدنا لهناك.. خلطه مشاعر انسانيه حقيقيه لدرجه غريبه.. دراما نقلته من كاتب شاب بيستعرض لصنايعي حقيقي تثق فيما س

sadly even in the sample original text there was no Arabic diacritical marks

In [10]:
print('oringinal text')
print(df.loc[:3, 'text'].to_list())


df['text'] = df['text'].apply(lambda x: simple_word_tokenize(x))
print('text after splitting')
print(df.loc[:3, 'text'].to_list())

oringinal text
['ممتاز نوعا ما . النظافه والموقع والتجهيز والشاطيء. المطعم', 'احد اسباب نجاح الامارات ان كل شخص في هذه الدوله يعشق ترابها. نحن نحب الامارات. ومضات من فكر. نصائح لدوله تطمح بالصفوف الاولي و قائد لا يقبل الا براحه شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءه مرات كثيره', 'هادفه .. وقويه. تنقلك من صخب شوارع القاهره الي هدوء جبال الشيشان .. للتعرف علي حقيقه ما يجري في تلك البلاد من حروب ضاربه بحق المسلمين و جزء كبير من تاريخ تلك المنطقه. التضحيه .. الرجوله .. الوفاء والكثير من القيم الاخري اثبتت وجودها في تلك الروايه البسيطه', 'خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطي مرحله ان القارئ يخلص الروايه وهو فاتح بؤه لمرحله ان القارئ يخلص الروايه وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبه ف اوائل القرن العشرين وجه ياخدنا لهناك.. خلطه مشاعر انسانيه حقيقيه لدرجه غريبه.. دراما نقلته من كاتب شاب بيستعرض لصنايعي حقيقي تثق فيما س

In [11]:
words = df['text'].tolist()
words = [item for sublist in words for item in sublist]

words_counter = Counter(words)
words_set = sorted(Counter(words),key=Counter(words).get,reverse=True)
vocab_to_int = []

vocab_to_int = {word: index for index,word in enumerate(words_set, 1)  if words_counter[word] > 10}

print(f'length of the vocab is {len(vocab_to_int)}')
# vocab_to_int = {word: index for (index,word),stop in zip(enumerate(words_set, 1), range(50000))}
# vocab_to_int = {word: index for index,word in enumerate(words_set, 1)}

length of the vocab is 35031


In [12]:
#printing the highest 20 words mentioned in the data
for i,stop in zip(vocab_to_int.items(), range(20)):
    print(i)

('.', 1)
('في', 2)
('من', 3)
('،', 4)
('و', 5)
('ان', 6)
('علي', 7)
('لا', 8)
('ما', 9)
('عن', 10)
('جدا', 11)
('لم', 12)
('الي', 13)
('الروايه', 14)
('الكتاب', 15)
('كل', 16)
('كان', 17)
('التي', 18)
(':', 19)
('هذا', 20)


now let's tokenize the text data to ints.

In [13]:
reviews = df['text'].tolist()
reviews_ints = []
for review in reviews:
    reviews_ints.append([vocab_to_int.get(word, 0) for word in review])
print(reviews_ints[:3])

[[78, 320, 9, 1, 95, 1174, 0, 29330, 1, 289], [135, 1190, 2647, 1586, 6, 16, 265, 2, 30, 606, 6948, 0, 1, 381, 3053, 1586, 1, 15054, 3, 1311, 1, 3720, 18759, 32838, 0, 176, 5, 7646, 8, 4262, 36, 7389, 9467, 3991, 16, 12834, 2115, 1954, 1, 1157, 5, 187, 20050, 308, 2887, 1676, 70, 2465, 41, 2527, 58, 55, 208, 1, 221, 152, 796, 293], [10743, 1, 1, 14354, 1, 10914, 3, 10574, 5901, 1873, 13, 1573, 15055, 19379, 1, 1, 8595, 7, 401, 9, 4263, 2, 86, 1564, 3, 6792, 32839, 1337, 646, 5, 359, 198, 3, 433, 86, 1392, 1, 6659, 1, 1, 15056, 1, 1, 7647, 2204, 3, 2308, 426, 9323, 3489, 2, 86, 14, 1775]]


In [14]:
labels = df['label'].tolist()
encoded_labels = []
for label in labels:
    if label == 'Positive':
        encoded_labels.append(2)
    if label == 'Neutral':
        encoded_labels.append(1)
    elif label == 'Negative':
        encoded_labels.append(0) 

unique, counts = np.unique(encoded_labels, return_counts=True)
distribution = dict(zip(unique, counts))
print('encoded_labels class distribution:')
for item in distribution.items():
    print(item)

encoded_labels class distribution:
(0, 33333)
(1, 33333)
(2, 33333)


In [15]:
features = pad_features(reviews_ints, 256)

In [16]:
print(features.shape)

(99999, 256)


In [17]:
train_x, validate_x, test_x, train_y, validate_y, test_y = train_test_split(features, encoded_labels, numclasses=3, train_frac=0.8, balanced=True)

make sure the classes (labels) are sorted to have a balanced splits


In [18]:
unique, counts = np.unique(train_y, return_counts=True)
distribution = dict(zip(unique, counts))
print('train_y class distribution:')
for item in distribution.items():
    print(item)

unique, counts = np.unique(validate_y, return_counts=True)
distribution = dict(zip(unique, counts))
print('validate_y class distribution:')
for item in distribution.items():
    print(item)

unique, counts = np.unique(test_y, return_counts=True)
distribution = dict(zip(unique, counts))
print('test_y class distribution:')
for item in distribution.items():
    print(item)

train_y class distribution:
(0, 26666)
(1, 26667)
(2, 26667)
validate_y class distribution:
(0, 3333)
(1, 3333)
(2, 3333)
test_y class distribution:
(0, 3333)
(1, 3333)
(2, 3333)


In [19]:
print(train_x.shape, train_y.shape)
print(validate_x.shape, validate_y.shape)
print(test_x.shape, test_y.shape)

(80000, 256) (80000,)
(9999, 256) (9999,)
(9999, 256) (9999,)


In [20]:
#creating data loaders to be used in training and testing 
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(validate_x), torch.from_numpy(validate_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

#batch_size for dataloaders
batch_size = 256

num_worker = 0

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True, num_workers=num_worker)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, drop_last=True, num_workers=num_worker)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True, num_workers=num_worker)

since the dataset are not very large let's try a few architecture, while doing a simple grid search for each architecture optimal learning rate

In [56]:
lr_list = [
    1e-2, 2e-2, 3e-2, 4e-2, 5e-2, 6e-2, 7e-2, 8e-2, 9e-2,
    1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3, 9e-3
]
# lr_list = [
#     1e-2, 5e-2, 1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5, 1e-6,
# ]
for lr in lr_list:
    torch.cuda.empty_cache()    
    #make sure the gpu is avalibale before moving the data to It else the work would be done using the cpu
    train_on_gpu=torch.cuda.is_available()

    #specifing the model hypermetrs and creating the model
    vocab_size = len(vocab_to_int)+1
    output_size = 3
    embedding_dim = 300
    hidden_dim = 200
    n_layers = 2
    
    net = multi_class_model.SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, train_on_gpu=train_on_gpu,
                    bidirectional=False, drop_prob=0.5)

    print(net)

    # optimization functions
    # optimizer = torch.optim.Adam(net.parameters(), lr=lr,weight_decay=0.00001)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    print(f'training with learning rate of {lr}....')
    # training the model
    train(net, train_loader, valid_loader, optimizer, vocab_to_int, batch_size, multi_class=True, seq_length=256, epochs=10,
        print_every=1, train_on_gpu=train_on_gpu, save_dic='models/arabic_01.pth')
    print('\n\n\n\n')
    torch.cuda.empty_cache()

SentimentRNN(
  (embd): Embedding(35032, 300)
  (lstm): LSTM(300, 200, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=200, out_features=3, bias=True)
  (LogSoftmax): LogSoftmax(dim=1)
)
training with learning rate of 0.01....
Epoch: 1/10... Step: 312... Loss: 1.099413... Val Loss: 1.098823
Validation loss decreased (inf --> 1.098823).  Saving model ...
Epoch: 2/10... Step: 624... Loss: 1.082270... Val Loss: 1.079725
Validation loss decreased (1.098823 --> 1.079725).  Saving model ...
Epoch: 3/10... Step: 936... Loss: 1.066272... Val Loss: 0.999108
Validation loss decreased (1.079725 --> 0.999108).  Saving model ...
Epoch: 4/10... Step: 1248... Loss: 0.899566... Val Loss: 0.978857
Validation loss decreased (0.999108 --> 0.978857).  Saving model ...
Epoch: 5/10... Step: 1560... Loss: 0.819070... Val Loss: 0.851041
Validation loss decreased (0.978857 --> 0.851041).  Saving model ...
Epoch: 6/10... Step: 1872... Loss: 0.7

In [57]:
torch.cuda.empty_cache()    
# make sure the gpu is avalibale before moving the data to It else the work would be done using the cpu
train_on_gpu=torch.cuda.is_available()

# specifing the model hypermetrs and creating the model
vocab_size = len(vocab_to_int)+1
output_size = 3
embedding_dim = 300
hidden_dim = 200
n_layers = 2

net = multi_class_model.SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, train_on_gpu=train_on_gpu,
            bidirectional=False, drop_prob=0.5)

print(net)

lr = 5e-3
# optimizer = torch.optim.Adam(net.parameters(), lr=lr,weight_decay=0.00001)
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

SentimentRNN(
  (embd): Embedding(35032, 300)
  (lstm): LSTM(300, 200, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=200, out_features=3, bias=True)
  (LogSoftmax): LogSoftmax(dim=1)
)


In [58]:
# training the model
train(net, train_loader, valid_loader, optimizer, vocab_to_int, batch_size, multi_class=True, seq_length=256, epochs=10,
print_every=1, train_on_gpu=train_on_gpu, save_dic='models/arabic_01.pth')
torch.cuda.empty_cache()

Epoch: 1/10... Step: 312... Loss: 1.097829... Val Loss: 1.098659
Validation loss decreased (inf --> 1.098659).  Saving model ...
Epoch: 2/10... Step: 624... Loss: 0.954658... Val Loss: 0.994099
Validation loss decreased (1.098659 --> 0.994099).  Saving model ...
Epoch: 3/10... Step: 936... Loss: 0.904346... Val Loss: 0.869085
Validation loss decreased (0.994099 --> 0.869085).  Saving model ...
Epoch: 4/10... Step: 1248... Loss: 0.980943... Val Loss: 0.964746
Epoch: 5/10... Step: 1560... Loss: 0.826960... Val Loss: 0.805186
Validation loss decreased (0.869085 --> 0.805186).  Saving model ...
Epoch: 6/10... Step: 1872... Loss: 0.690083... Val Loss: 0.817309
Epoch: 7/10... Step: 2184... Loss: 0.638771... Val Loss: 0.796134
Validation loss decreased (0.805186 --> 0.796134).  Saving model ...
Epoch: 8/10... Step: 2496... Loss: 0.708226... Val Loss: 0.773453
Validation loss decreased (0.796134 --> 0.773453).  Saving model ...
Epoch: 9/10... Step: 2808... Loss: 0.626841... Val Loss: 0.800724


In [59]:
# testing the model
test(test_loader, train_on_gpu, batch_size, multi_class=True, net=net)

Test loss: 0.770
Test accuracy: 0.650


In [60]:
# testing the last saved model
test(test_loader, train_on_gpu, batch_size, multi_class=True, model_path='models/arabic_01.pth')

Test loss: 0.775
Test accuracy: 0.646


In [26]:
# lr_list = [
#     1e-2, 2e-2, 3e-2, 4e-2, 5e-2, 6e-2, 7e-2, 8e-2, 9e-2,
#     1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 6e-3, 7e-3, 8e-3, 9e-3
# ]
lr_list = [
    1e-2, 5e-2, 1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5, 1e-6,
]
for lr in lr_list:
    torch.cuda.empty_cache()    
    #make sure the gpu is avalibale before moving the data to It else the work would be done using the cpu
    train_on_gpu=torch.cuda.is_available()

    #specifing the model hypermetrs and creating the model
    vocab_size = len(vocab_to_int)+1
    output_size = 3
    embedding_dim = 200
    hidden_dim = 150
    n_layers = 1
    
    net = multi_class_model.SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, train_on_gpu=train_on_gpu,
                    bidirectional=False, drop_prob=0)

    print(net)

    # optimization functions
    # optimizer = torch.optim.Adam(net.parameters(), lr=lr,weight_decay=0.00001)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    print(f'training with learning rate of {lr}....')
    # training the model
    train(net, train_loader, valid_loader, optimizer, vocab_to_int, batch_size, multi_class=True, seq_length=256, epochs=10,
        print_every=1, train_on_gpu=train_on_gpu, save_dic='models/arabic_01.pth')
    print('\n\n\n\n')
    torch.cuda.empty_cache()

SentimentRNN(
  (embd): Embedding(35032, 200)
  (lstm): LSTM(200, 150, batch_first=True)
  (dropout): Dropout(p=0, inplace=False)
  (fc1): Linear(in_features=150, out_features=3, bias=True)
  (LogSoftmax): LogSoftmax(dim=1)
)
training with learning rate of 0.01....
Epoch: 1/10... Step: 312... Loss: 1.099513... Val Loss: 1.099207
Validation loss decreased (inf --> 1.099207).  Saving model ...
Epoch: 2/10... Step: 624... Loss: 1.056846... Val Loss: 1.046425
Validation loss decreased (1.099207 --> 1.046425).  Saving model ...
Epoch: 3/10... Step: 936... Loss: 0.963622... Val Loss: 0.955144
Validation loss decreased (1.046425 --> 0.955144).  Saving model ...
Epoch: 4/10... Step: 1248... Loss: 0.832367... Val Loss: 0.953540
Validation loss decreased (0.955144 --> 0.953540).  Saving model ...
Epoch: 5/10... Step: 1560... Loss: 0.880639... Val Loss: 0.929750
Validation loss decreased (0.953540 --> 0.929750).  Saving model ...
Epoch: 6/10... Step: 1872... Loss: 0.835394... Val Loss: 0.940484
E

In [47]:
torch.cuda.empty_cache()    
# make sure the gpu is avalibale before moving the data to It else the work would be done using the cpu
train_on_gpu=torch.cuda.is_available()

# specifing the model hypermetrs and creating the model
vocab_size = len(vocab_to_int)+1
output_size = 3
embedding_dim = 300
hidden_dim = 200
n_layers = 2

net = multi_class_model.SentimentRNN(
    vocab_size, output_size, embedding_dim, hidden_dim, n_layers,
    train_on_gpu=train_on_gpu, bidirectional=False, drop_prob=0.5
    )

print(net)

lr = 5e-3
# optimizer = torch.optim.Adam(net.parameters(), lr=lr,weight_decay=0.00001)
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

SentimentRNN(
  (embd): Embedding(35032, 300)
  (lstm): LSTM(300, 200, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=400, out_features=3, bias=True)
  (LogSoftmax): LogSoftmax(dim=1)
)


In [48]:
# training the model
train(net, train_loader, valid_loader, optimizer, vocab_to_int, batch_size, multi_class=True, seq_length=256, epochs=10,
print_every=1, train_on_gpu=train_on_gpu, save_dic='models/arabic_01.pth')
torch.cuda.empty_cache()

Epoch: 1/10... Step: 312... Loss: 1.098610... Val Loss: 1.098869
Validation loss decreased (inf --> 1.098869).  Saving model ...
Epoch: 2/10... Step: 624... Loss: 1.097386... Val Loss: 1.099022
Epoch: 3/10... Step: 936... Loss: 0.881335... Val Loss: 0.940504
Validation loss decreased (1.098869 --> 0.940504).  Saving model ...
Epoch: 4/10... Step: 1248... Loss: 0.870823... Val Loss: 0.830931
Validation loss decreased (0.940504 --> 0.830931).  Saving model ...
Epoch: 5/10... Step: 1560... Loss: 0.772145... Val Loss: 0.805867
Validation loss decreased (0.830931 --> 0.805867).  Saving model ...
Epoch: 6/10... Step: 1872... Loss: 0.686307... Val Loss: 0.763641
Validation loss decreased (0.805867 --> 0.763641).  Saving model ...
Epoch: 7/10... Step: 2184... Loss: 0.606457... Val Loss: 0.751648
Validation loss decreased (0.763641 --> 0.751648).  Saving model ...
Epoch: 8/10... Step: 2496... Loss: 0.612857... Val Loss: 0.763980
Epoch: 9/10... Step: 2808... Loss: 0.499376... Val Loss: 0.770630


In [49]:
# testing the model
test(test_loader, train_on_gpu, batch_size, multi_class=True, net=net)

Test loss: 0.807
Test accuracy: 0.651


In [50]:
# testing the last saved model
test(test_loader, train_on_gpu, batch_size, multi_class=True, model_path='models/arabic_01.pth')

Test loss: 0.769
Test accuracy: 0.650
